### Problem
You need to process items in an iterable, but for whatever reason, you can't or don't want to use a for loop.

In [2]:
# Use the next() function and StopIteration exception
with open('etc/passwd') as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration:
        pass

hello
12312312
13343
12343


In [3]:
with open('etc/passwd') as f:
    while True:
        line = next(f, None)
        if line is None:
            break
        print(line, end='')

hello
12312312
13343
12343


In [1]:
# What happens during iteration
items = [1, 2, 3]

In [2]:
# Get the iterator
it = iter(items)

In [3]:
# Run the iterator
next(it)

1

In [4]:
next(it)

2

In [5]:
next(it)

3

### Problem
You have built a custom container object that internally holds a list, tuple, or some other iterable. You would like to make iteration work with your new container.

In [9]:
# You need a __iter__ method in your class
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)

In [11]:
# Example
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    
    for ch in root:
        print(ch)

Node(1)
Node(2)


### Problem
You want to implement a custom iteration pattern that's different than the usual built-in functions (e.g. range(), reversed(), etc.)

In [12]:
# Define it using a generator function
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

In [13]:
for n in frange(0, 4, 0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [14]:
list(frange(0, 1, 0.125))

[0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]

In [15]:
# How it works
def countdown(n):
    print('Starting to count from ', n)
    while n > 0:
        yield n
        n -= 1
    print("Done!")

In [16]:
# Create the generator, notice no output appears
c = countdown(3)
c

<generator object countdown at 0x7fa6cc2a9a98>

In [17]:
# Run to first yield and emit a value
next(c)

Starting to count from  3


3

In [18]:
# Run to the next yield
next(c)

2

In [19]:
# Run to next yield
next(c)

1

In [20]:
# Run to next yield (iteration stops)
next(c)

Done!


StopIteration: 

### Problem
You are building custom objects on which you would like to support iteration, but would like an easy way to implement the iterator protocol.

In [21]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

In [22]:
# Example
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In [23]:
# Alternative implementation of the depth_first method
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, other_node):
        self._children.append(other_node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        return DepthFirstIterator(self)
    
class DepthFirstIterator(object):
    '''
    Depth-first traversal
    '''
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
        
    def __iter__(self):
        return self
    
    def __next__(self):
        # Return myself if just started; create an iterator for children
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        
        # If processing a child, return its next item
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
            
        # Advance to the next child and start its iteration
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)

### Problem
You want to iterate in reverse over a sequence.

In [24]:
# Use the built-in reversed() function
a = [1, 2, 3, 4]

In [25]:
for x in reversed(a):
    print(x)

4
3
2
1


In [26]:
for x in a[::-1]:
    print(x)

4
3
2
1


In [27]:
# Convert to a list if necessary
f = open('etc/passwd')
for line in reversed(list(f)):
    print(line, end='')

12343
13343
12312312
hello


In [28]:
# Reversed iteration can be customized
class Countdown:
    def __init__(self, start):
        self.start = start
    
    # Forward iterator
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
            
    # Reverse iterator
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n
            n += 1

### Problem
You would like to define a generator function, but it involves extra state that you would like to expose to the user somehow.

In [29]:
# Implement it as a class
from collections import deque

class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for lineno, line in enumerate(self.lines, 1):
            self.history.append((lineno, line))
            yield line
            
    def clear(self):
        self.history.clear()

In [30]:
with open('etc/passwd') as f:
    lines = linehistory(f)
    for line in lines:
        if 'python' in line:
            for lineno, hline in lines.history:
                print('{}:{}'.format(lineno, hline), end='')

In [31]:
f = open('etc/passwd')
lines = linehistory(f)

In [32]:
next(lines)

TypeError: 'linehistory' object is not an iterator

In [33]:
# Call iter() first, then start iterating
it = iter(lines)

In [34]:
next(it)

'hello\n'

In [35]:
next(it)

'12312312\n'

### Problem
You want to take a slice of data produced by an iterator, but the normal slicing operator doesn't work.

In [37]:
# Using itertools.islice()
def count(n):
    while True:
        yield n
        n += 1

In [43]:
c = count(0)

In [39]:
c[10:20]

TypeError: 'generator' object is not subscriptable

In [44]:
# Now using islice()
import itertools
for x in itertools.islice(c, 10, 20):
    print(x)

10
11
12
13
14
15
16
17
18
19


In [45]:
# It discards the previous items and starts the slice from the next number
for x in itertools.islice(c, 10, 20):
    print(x)

30
31
32
33
34
35
36
37
38
39


### Problem
You want to iterate over items in an iterable, but the first few items aren't of interest and you just want to discard them.

In [47]:
# Using itertools.dropwhile()
with open('/etc/passwd') as f:
    for line in f:
        print(line, end='')

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
systemd-network:x:100:102:systemd Network Management,,,:/run/systemd:/usr/sbin/nologin
systemd-resolve:x:101:103:systemd Re

In [48]:
# If you want to skip all the initial comment lines
from itertools import dropwhile
with open('/etc/passwd') as f:
    for line in dropwhile(lambda line: line.startswith('#'), f):
        print(line, end='')

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
systemd-network:x:100:102:systemd Network Management,,,:/run/systemd:/usr/sbin/nologin
systemd-resolve:x:101:103:systemd Re

In [49]:
# You can use islice if you know exactly the number of lines to skip
from itertools import islice
items = ['a', 'b', 'c', 1, 4, 10, 15]
for x in islice(items, 3, None):
    print(x)

1
4
10
15


Discarding the first part of an iterable is also slightly different than simply filtering all
of it. For example, the first part of this recipe might be rewritten as follows:

In [54]:
with open('/etc/passwd') as f:
    lines = (line for line in f if not line.startswith('#'))
    
    for line in lines:
        print(line, end='')

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
systemd-network:x:100:102:systemd Network Management,,,:/run/systemd:/usr/sbin/nologin
systemd-resolve:x:101:103:systemd Re

This will obviously discard the comment lines at the start, but will also discard all such
lines throughout the entire file. On the other hand, the solution only discards items
until an item no longer satisfies the supplied test. After that, all subsequent items are
returned with no filtering.

### Problem
You want to iterate over all of the possible combinations or permutations of a collection of items.

In [55]:
# Using itertools permutations
items = ['a', 'b', 'c']
from itertools import permutations
for p in permutations(items):
    print(p)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


In [56]:
# Permutations of certain length
for p in permutations(items, 2):
    print(p)

('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')


In [57]:
# Using itertools.combinations()

In [58]:
from itertools import combinations
for c in combinations(items, 3):
    print(c)

('a', 'b', 'c')


In [59]:
for c in combinations(items, 2):
    print(c)

('a', 'b')
('a', 'c')
('b', 'c')


In [60]:
for c in combinations(items, 1):
    print(c)

('a',)
('b',)
('c',)


In [65]:
# To choose the same item more than once using itertools.combinations_with_replacement
from itertools import combinations_with_replacement
for c in combinations_with_replacement(items, 3):
    print(c)

('a', 'a', 'a')
('a', 'a', 'b')
('a', 'a', 'c')
('a', 'b', 'b')
('a', 'b', 'c')
('a', 'c', 'c')
('b', 'b', 'b')
('b', 'b', 'c')
('b', 'c', 'c')
('c', 'c', 'c')


### Problem
You want to iterate over a sequence, but would like to keep track of which element of the sequence is currently being processed.

In [66]:
# Using enumerate
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list):
    print(idx, val)

0 a
1 b
2 c


In [68]:
# For using canonical numbers(starting with 1)
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list, 1):
    print(idx, val)

1 a
2 b
3 c


In [69]:
# Useful for tracking numbers in files you want to use a line number in an error message
def parse_data(filename):
    with open(filename, 'rt') as f:
        for lineno, line in enumerate(f, 1):
            fields = line.split()
            try:
                count = int(fields[1])
                ...
            except ValueError as e:
                print('Line {}: Parse error: {}'.format(lineno, e))

In [71]:
from collections import defaultdict
word_summary = defaultdict(list)

with open('etc/passwd', 'r') as f:
    lines = f.readlines()
    
for idx, line in enumerate(lines):
    # Create a list of words in current line
    words = [w.strip().lower() for w in line.split()]
    for word in words:
        word_summary[word].append(idx)

In [72]:
# Using enumerate() with a sequence of tuples
data = [(1, 2), (3, 4), (5, 6), (7, 8)]

# Correct
for n, (x, y) in enumerate(data):
    print(n, x, y)

0 1 2
1 3 4
2 5 6
3 7 8


In [73]:
# Error
for n, x, y in enumerate(data):
    print(n, x, y)

ValueError: not enough values to unpack (expected 3, got 2)